In [2]:
import utils        #need to import functions to use them 
import pandas as pd 
import os

In [ ]:
file_name = 'Atypical_heavy_light_chains.csv'
df = pd.read_csv(file_name)
df = utils.remove_rows(df, 'productive', True)
utils.amino_acid_count(df, 'R')
utils.crd3_pKa(df)
gene_counts_df = utils.get_gene_counts_df(df, ['v_gene', 'j_gene', 'd_gene'])
trim_df = utils.remove_by_threshold(df, gene_counts_df, 0.05)
#tgene_counts_df = utils.get_gene_counts_df(trim_df, ['v_gene', 'j_gene', 'd_gene'])
all_igh, all_iglk = utils.all_heavyLight_genes([trim_df], ['v_gene', 'j_gene', 'd_gene'])
for cell_type, cell_info in trim_df.groupby("Celltype (Cluster)"):
    pair_df = utils.get_pair_counts(cell_info, ['v_gene', 'j_gene', 'd_gene'])
    cur_counts_df = utils.get_gene_counts_df(cell_info, ['v_gene', 'j_gene', 'd_gene'])
    cur_all_igh, cur_all_iglk = utils.all_heavyLight_genes([cell_info], ['v_gene', 'j_gene', 'd_gene'])
    #GAHHH WHICH ONES TO USE HERE!
    output_df = utils.get_hypergoemetric_test(pair_df, cur_counts_df, all_igh, all_iglk)
    print(output_df)
    break

The following light chain genes were removed: ['IGKV2D-30', 'IGKV3D-7', 'IGKV3OR2-268', 'IGKV6D-41', 'IGLV3-12', 'IGLV5-52', 'IGLV2-8', 'IGLJ6']
           IGKL         IGH  observed  expected         p_val     adj_pvals
0     IGKV3D-11    IGHD1-14         0  0.002344  1.000000e+00  1.000000e+00
1     IGKV3D-11    IGHV3-13         0  0.000558  1.000000e+00  1.000000e+00
2     IGKV3D-11  IGHV1-69-2         0  0.006920  1.000000e+00  1.000000e+00
3     IGKV3D-11    IGHD4-23         0  0.003125  1.000000e+00  1.000000e+00
4     IGKV3D-11     IGHD2-2         0  0.008371  1.000000e+00  1.000000e+00
...         ...         ...       ...       ...           ...           ...
4923      IGKJ4    IGHV3-73         1  0.358929  3.046385e-01  7.298449e-01
4924      IGKJ4    IGHV1-24         3  0.740290  3.705028e-02  1.438801e-01
4925      IGKJ4    IGHD1-20         3  0.224330  1.188004e-03  9.889328e-03
4926      IGKJ4       IGHJ5        21  3.791183  1.478820e-10  9.588978e-09
4927      IGKJ4    

In [4]:
all_igh, all_iglk = utils.all_heavyLight_genes([trim_df], ['v_gene', 'j_gene', 'd_gene'])
needed_Lgenes = [gene for gene in all_iglk if gene.startswith("IGLV") or gene.startswith("IGKV")]
needed_Lgenes.sort()
trim_totals = []
for cell_type, cell_info in trim_df.groupby("Celltype (Cluster)"):
    cell_df = utils.lightChain_vs_heavyRcount_relFrequency(cell_info, needed_Lgenes, "cdr_R_count")
    trim_totals.append([cell_type, cell_df])

Organizing data from native files. Want to make graphs showing the relative frequenices of heavy chains with 0 Rs and with more the 0 Rs for each light chain (for each paitent: 3 hcd and 3 sle). 

In [2]:
directory = r"C:\Users\tskal\OneDrive\Desktop\Working for Mama!\files naive"
file_names = []
fkeys = []
for path, folders, files in os.walk(directory):
    for filename in files: 
        full_name = "files naive/"+filename
        fkey = filename[:3]+filename[4]
        file_names.append(full_name)
        fkeys.append(fkey)
dfs = []
for f in file_names:
    dfs.append(pd.read_csv(f))
all_igh, all_iglk = utils.all_heavyLight_genes(dfs, ['v_gene', 'j_gene', 'd_gene'])

In [3]:
needed_genes = [gene for gene in all_iglk if gene.startswith("IGLV") or gene.startswith("IGKV")]
needed_genes.sort()

In [ ]:
count = 0
needed_info = []
trimed_dfs = []
for cur_df in dfs: 
    #removing rows that are not productive or Naive 
    cur_df = utils.remove_rows(cur_df, 'productive', True)
    cur_df = utils.remove_rows(cur_df, 'is_naive', "Naive")
    #reoganizing df so that all information for on cell in on same row
    cur_grouped_df = utils.make_combined_rows(cur_df, ['IGH', 'IGK', 'IGL'])
    cur_gene_counts_df = utils.get_gene_counts_df(cur_grouped_df, ['v_gene', 'j_gene', 'd_gene'])
    trim_df = utils.remove_by_threshold(cur_grouped_df, cur_gene_counts_df, 0.05)
    #adding additional columns 
    utils.amino_acid_count(trim_df, 'R')
    utils.crd3_pKa(trim_df)
    #getting relative frequenices of heavy chain R amounts
    light_rcount_df = utils.lightChain_vs_heavyRcount_relFrequency(trim_df, needed_genes, "IGH_R_count")
    utils.add_ztest_values(light_rcount_df, ['Rcount_gt_0', 'Rcount_equalTo_0'], 'total')
    needed_info.append([fkeys[count], light_rcount_df])
    #PCIK ONE!
    dfs[count] = trim_df
    trimed_dfs.append(trim_df)
    count += 1
print(needed_info[0][1].columns)

The following light chain genes were removed: ['IGLV4-60', 'IGKV6D-21', 'IGKV3D-11', 'IGLV9-49', 'IGLJ7', 'IGKV1-39', 'IGLV5-37', 'IGLV8-61']
The following light chain genes were removed: ['IGKV3D-11', 'IGKV1D-13', 'IGKV2D-28', 'IGLV3-22']
The following light chain genes were removed: ['IGKV3D-7', 'IGKV1D-43', 'IGKV2D-26', 'IGLV4-3']
The following light chain genes were removed: ['IGKV2D-28', 'IGKV3D-11', 'IGKV1D-16', 'IGKV5-2', 'IGLV2-18', 'IGLV5-37', 'IGLJ7']
The following light chain genes were removed: ['IGKV2D-26', 'IGKV3D-11', 'IGKV5-2', 'IGLV5-39']
The following light chain genes were removed: ['IGKV3D-20', 'IGLV5-45', 'IGKV2-18', 'IGKV6D-21', 'IGKV1D-16', 'IGLV5-52']
Index(['light_gene', 'Rcount_equalTo_0', 'Rcount_gt_0', 'total', 'R_gt_0_freq',
       'R_equal_0_freq', 'zstat', 'p_value', 'significant'],
      dtype='object', name='Rcount_type')


In [6]:
h_total_df = needed_info[0][1].copy()
h_total_df = h_total_df.drop(columns=['total', 'R_gt_0_freq','R_equal_0_freq', 'zstat', 'p_value', 'significant'])
h_total_df["Rcount_equalTo_0"] += needed_info[1][1]['Rcount_equalTo_0'] + needed_info[2][1]['Rcount_equalTo_0'] 
h_total_df["Rcount_gt_0"] += needed_info[1][1]['Rcount_gt_0'] + needed_info[2][1]['Rcount_gt_0'] 
h_total_df["Total"] = h_total_df["Rcount_equalTo_0"] + h_total_df["Rcount_gt_0"]
h_total_df['R_gt_0_freq'] = h_total_df['Rcount_gt_0']/h_total_df['Total'] *100
h_total_df['R_equal_0_freq'] = h_total_df['Rcount_equalTo_0']/h_total_df['Total'] *100
utils.add_ztest_values(h_total_df, ['Rcount_gt_0', 'Rcount_equalTo_0'], 'Total')

s_total_df = needed_info[3][1].copy()
s_total_df = s_total_df.drop(columns=['total', 'R_gt_0_freq','R_equal_0_freq', 'zstat', 'p_value', 'significant'])
s_total_df["Rcount_equalTo_0"] += needed_info[4][1]['Rcount_equalTo_0'] + needed_info[5][1]['Rcount_equalTo_0'] 
s_total_df["Rcount_gt_0"] += needed_info[4][1]['Rcount_gt_0'] + needed_info[5][1]['Rcount_gt_0'] 
s_total_df["Total"] = s_total_df["Rcount_equalTo_0"] + s_total_df["Rcount_gt_0"]
s_total_df['R_gt_0_freq'] = s_total_df['Rcount_gt_0']/s_total_df['Total'] *100
s_total_df['R_equal_0_freq'] = s_total_df['Rcount_equalTo_0']/s_total_df['Total'] *100
utils.add_ztest_values(s_total_df, ['Rcount_gt_0', 'Rcount_equalTo_0'], 'Total')

excel_graph_format = {
    "Individual": {
        "tables": [{"data": info[1], "title": f"{info[0]} Heavy Chain R Amount for Each Light Chain",
                "graph_type": "Bar","graph_name": info[0],
                "graph_details": {"x": "light_gene", "y": ["R_gt_0_freq", "R_equal_0_freq"]}}
            for info in needed_info]}, 
    "Totals": {"tables":[
        {"data": h_total_df, "title": f"Total hcd Heavy Chain R Amount for Each Light Chain", "graph_type": "Bar","graph_name": "hcd Totals", 
         "graph_details": {"x": "light_gene", "y": ["R_gt_0_freq", "R_equal_0_freq"]}}, 
        {"data": s_total_df, "title": f"Total sle Heavy Chain R Amount for Each Light Chain", "graph_type": "Bar","graph_name": "sle Totals",
        "graph_details": {"x": "light_gene", "y": ["R_gt_0_freq", "R_equal_0_freq"]}}
    ]}
}   
excel_format = {
    "Individual": {
        "tables": [{"data": info[1], "title": f"{info[0]} Heavy Chain R Amount for Each Light Chain"}
            for info in needed_info]}, 
    "Totals": {"tables":[
        {"data": h_total_df, "title": f"Total hcd Heavy Chain R Amount for Each Light Chain"}, 
        {"data": s_total_df, "title": f"Total sle Heavy Chain R Amount for Each Light Chain"}
    ]}
}   
utils.all_to_excel(excel_format, "naive_R_stuff_ztestNums_alt")

Writing sheet: Individual
Writing sheet: Totals


Doing same analysis but for different dataset (and splitting by cell type this time)!

In [24]:
f_name = "Atypical_heavy_light_chains.csv"
g_df = pd.read_csv(f_name)
all_igh, all_iglk = utils.all_heavyLight_genes([g_df], ['v_gene', 'j_gene', 'd_gene'])
g_df = utils.remove_rows(g_df, 'productive', True)
#adding additional columns 
utils.amino_acid_count(g_df, 'R')
utils.crd3_pKa(g_df)
needed_Lgenes = [gene for gene in all_iglk if gene.startswith("IGLV") or gene.startswith("IGKV")]
needed_Lgenes.sort()
processed_samples = [] #[[name, [cell type, heavy/light R freq], [cell type, heavy/light R freq], ...], [name, ...]]
for name, sample_info in g_df.groupby("Sample"):
    sample = [name]
    for cell_type, cell_info in sample_info.groupby("Celltype (Cluster)"):
        cell_df = utils.lightChain_vs_heavyRcount_relFrequency(cell_info, needed_Lgenes, "cdr_R_count")
        sample.append([cell_type, cell_df])
    processed_samples.append(sample)

In [28]:
totals = []
for cell_type, cell_info in g_df.groupby("Celltype (Cluster)"):
    cell_df = utils.lightChain_vs_heavyRcount_relFrequency(cell_info, needed_Lgenes, "cdr_R_count")
    totals.append([cell_type, cell_df])
e_format = {}
for sample in processed_samples: 
    e_format[sample[0]] = {"tables": [{"data": info[1], "title": f"{info[0]} Heavy Chain R Amount for Each Light Chain",
                "graph_type": "Bar","graph_name": info[0],
                "graph_details": {"x": "light_gene", "y": ["R_gt_0_freq", "R_equal_0_freq"]}}
            for info in sample[1:]]}
e_format["Total"] = {"tables": [{"data": info[1], "title": f"{info[0]} Heavy Chain R Amount for Each Light Chain",
                "graph_type": "Bar","graph_name": info[0],
                "graph_details": {"x": "light_gene", "y": ["R_gt_0_freq", "R_equal_0_freq"]}}
            for info in totals]}
print("Keys in e_format:", e_format.keys())
utils.all_to_excel(e_format, "R_analysis_Atypical_heavy_light_chains")

Keys in e_format: dict_keys(['Malaria-Exp 1', 'Malaria-Exp 2', 'Non-Exp 1', 'Non-Exp 2', 'Total'])
Writing sheet: Malaria-Exp 1
Writing sheet: Malaria-Exp 2
Writing sheet: Non-Exp 1
Writing sheet: Non-Exp 2
Writing sheet: Total
